In [1]:
# default_exp showdoc

# Showing Verbose Documentation

> Showing input documentation with nbdev

In [2]:
#hide
from nbverbose.showdoc import *

In [112]:
#export
from nbdev.showdoc import (
    _format_enum_doc, _format_cls_doc, 
    _format_func_doc, add_doc_links,
    get_source_link, qual_name, is_enum,
    get_doc_link, md2html
)

import re

import inspect

from fastcore.foundation import Config
from fastcore.imports import IN_COLAB
from fastcore.utils import IN_NOTEBOOK
from fastcore.docments import docments

if IN_NOTEBOOK:
    from IPython.display import Markdown,display
    from IPython.core import page

In [55]:
#export
def _format_args(
    elt, # A function or class
) -> str: # A formatted docstring for arguments
    "Generates a formatted argument string"
    ment_dict = docments(elt, full=True)
    ret = None
    arg_string = ""
    if "self" in ment_dict.keys(): ment_dict.pop("self")
    if "cls" in ment_dict.keys(): ment_dict.pop("cls")
    if "return" in ment_dict.keys(): 
        ret = ment_dict["return"]
        ment_dict.pop("return")
    if len(ment_dict.keys()) > 0: 
        arg_string += '|**Parameters:**|Type|Default|Details|\n|---|---|---|---|\n'
        for key, item in ment_dict.items():
            is_required=True
            if key == 'return': continue
            if item['default'] != inspect._empty:
                is_required = False
            arg_string += f"|**`{key}`**|"
            arg_string += "*None Specified*|" if item['anno'] == inspect._empty else f"`{item['anno']}`|"
            arg_string += "*No Default*|" if is_required else f"`{str(item['default'])}`|"
            if item['docment'] is not None:
                arg_string += f"{item['docment']}|"
            arg_string += '\n'
    return_string = ""
    if ret is not None:
        return_string += "|**Return Type**|Details|\n|-|-|\n|"
        if not ret['anno'] == inspect._empty:
            return_string += f"`{ret['anno']}`|"
        if "docment" in ret.keys():
            if ret['docment'] is not None:
                return_string += f"{ret['docment']}|"
    return arg_string + "\n\n" + return_string

In [56]:
#export
def show_doc(
    elt, # Some function or class to pull up the documentation for
    doc_string:bool=True, # Whether to display the doc string
    name:str=None, # An optional name to use instead
    title_level:int=None, # The heading level
    disp:bool=True, # Whether to display the Markdown
    default_cls_level:int=2 # If elt is a class, a heading level to use for it
) -> str: # The documentation as a string
    "Show documentation for element `elt` with potential verbose inputs. Supported types: class, function, and enum."
    elt = getattr(elt, '__func__', elt)
    qname = name or qual_name(elt)
    if inspect.isclass(elt):
        if is_enum(elt): name,args = _format_enum_doc(elt, qname)
        else:            name,args = _format_cls_doc (elt, qname)
    elif callable(elt):  name,args = _format_func_doc(elt, qname)
    else:                name,args = f"<code>{qname}</code>", ''
    link = get_source_link(elt)
    source_link = f'<a href="{link}" class="source_link" style="float:right">[source]</a>'
    title_level = title_level or (default_cls_level if inspect.isclass(elt) else 4)
    doc =  f'<h{title_level} id="{qname}" class="doc_header">{name}{source_link}</h{title_level}>'
    doc += f'\n\n> {args}\n\n' if len(args) > 0 else '\n\n'
    if doc_string and inspect.getdoc(elt):
        s = inspect.getdoc(elt)
        # show_doc is used by doc so should not rely on Config
        try: monospace = (Config().get('monospace_docstrings') == 'True')
        except: monospace = False
        # doc links don't work inside markdown pre/code blocks
        s = f'```\n{s}\n```' if monospace else add_doc_links(s, elt)
        doc += s
    if len(args) > 0:
        try:
            doc += f"\n\n{_format_args(elt)}"
        except:
            pass
    if disp: display(Markdown(doc))
    else: return doc

In [129]:
#export
_TABLE_CSS = """<style>
    table { border-collapse: collapse; border:thin solid #dddddd; margin: 25px 0px; ; }
    table tr:first-child { background-color: #FFF}
    table thead th { background-color: #eee; color: #000; text-align: center;}
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 5px; }
    tr:nth-child(even) {background: #eee;}</style>"""

In [142]:
#export
def doc(
    elt, # Some function or class to pull up the documentation for
):
    "Show `show_doc` info in preview window when used in a notebook"
    md = show_doc(elt, disp=False)
    doc_link = get_doc_link(elt)
    if doc_link is not None:
        md += f'\n\n<a href="{doc_link}" target="_blank" rel="noreferrer noopener">Show in docs</a>'
    output = md2html(md)
    if IN_COLAB: get_ipython().run_cell_magic(u'html', u'', output + _TABLE_CSS)
    else:
        try: page.page({'text/html': output + _TABLE_CSS})
        except: display(Markdown(md))